In [1]:

import os
# from keras.utils import multi_gpu_model
os.environ["CUDA_VISIBLE_EDVICES"] = '0, 2, 3'
from IPython import get_ipython
from tensorflow.keras import backend as K
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
# tf.compat.v1.disable_eager_execution()
from tensorflow.keras import layers
from tensorflow.keras import initializers

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



2024-07-01 19:47:02.486639: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-07-01 19:47:02.486663: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-01 19:47:04.036791: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 19:47:04.040106: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-01 19:47:04.041384: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Suc

In [95]:
import pandas as pd
from pandas import Series
import numpy as np
from numpy import polyfit
from matplotlib import pyplot
import matplotlib as mpl
# import seaborn as sns

from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
# from keras.utils import plot_model
from tensorflow.keras import optimizers

import time
import datetime
import math
import tensorflow.keras.callbacks
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error,r2_score
import tensorflow.keras.callbacks
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from scipy.stats.stats import pearsonr
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import load_model
import pickle
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
# from statsmodels.tsa.seasonal import seasonal_decompose
from typing import List


In [96]:
# import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [97]:

# #   TCN
boolean_train_TCN_model = True

# window size of input as predictors
window_size = 60 # use the past window_size days's sst to predict the next day's sst
ndays_predict_window_size = 30
pointNum=12

# degree of polynomial for fitting seasonality
seasonality_fit_degree=6

# fraction of training samples
train_sample_fraction=0.8

# LSTM model major version
major_version = 35
a_time = time.ctime()
location='0000'

In [98]:
# create a folder to store results of the location
# dir_data = '/home/ldd/data_code/Prediction/DET/results'

dir_data = '/0_SSS/'


# resultsDir_en = dir_data  + '/en_point1/' + str(a_time) +'/'  # English version

resultsDir_en = dir_data  + 'results/points' +str(pointNum) +'/'+ str(a_time) +'/'
resultsDir_cn = dir_data+ 'results/'  # Chinese version



each_training_records_dir=''


TCN_rm_seasonality_dir ='TCN_rm_seasonality_dir/'
#---

# 保存画图的地址
BoxPlot_dir_all = resultsDir_en+'30/'

# 保存实验结果的地址
BoxPlot_dir = resultsDir_en+'result/'

# 保存传统方法的地址
Tra_dir = '/0_SSS/tra_method/point'+ str(pointNum) +'/'


if not os.path.exists(Tra_dir):
    os.makedirs(Tra_dir)

if not os.path.exists(resultsDir_en):
    os.makedirs(resultsDir_en)

if not os.path.exists(resultsDir_cn):
    os.makedirs(resultsDir_cn)


# TCN
if not os.path.exists(resultsDir_en + TCN_rm_seasonality_dir):
    os.makedirs(resultsDir_en + TCN_rm_seasonality_dir)

if not os.path.exists(resultsDir_cn + TCN_rm_seasonality_dir):
    os.makedirs(resultsDir_cn + TCN_rm_seasonality_dir)

if not os.path.exists(resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir):
    os.makedirs(resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir)

if not os.path.exists(resultsDir_cn + TCN_rm_seasonality_dir + each_training_records_dir):
    os.makedirs(resultsDir_cn + TCN_rm_seasonality_dir + each_training_records_dir)





In [99]:

sst_file=dir_data+'point'+str(pointNum)+'_salinity_values.csv'
series=pd.read_csv(sst_file, header=None) # use sst anomalies
# dates=pd.read_csv(sst_file,usecols=[1], engine='python')

series.shape
print(series)

In [ ]:
# save descriptions of the observed SST series to csv file
series_description=series.describe()
print(series_description)
series_description.to_csv(resultsDir_en+'series_description.csv')


In [ ]:
# plot orignial time series (English version)
n_days=[(i+1) for i in range(len(series.values))]

fig,ax=pyplot.subplots(figsize=(15,6))
fig.suptitle('Daily salt',fontsize=18,fontweight='bold')

ax.plot(n_days,series.values,color='k')

# set x,y label
ax.set_xlabel(xlabel='number of days',
              fontsize=18,
              fontweight='bold')
ax.set_ylabel(ylabel='SSS(psu)',
              fontsize=18,
              fontweight='bold')
# set tick labels
pyplot.setp(ax.get_xticklabels(),fontsize=18)
pyplot.setp(ax.get_yticklabels(),fontsize=18)
# add grid
#ax.grid()
# add legend
#legend=ax.legend(loc='upper left',fontsize=15,ncol=2)
#legend=ax.legend(loc='upper left',fontsize=15, bbox_to_anchor=(0.825, 1.18))

# save fige
fig.savefig(resultsDir_en+'SSS'+'.eps',
            dpi=30,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')
# pyplot.show()

In [104]:
# normalize
series=pd.DataFrame(data=series)
series.describe()
print(series.shape)
# normalize features

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(series.values)
series = pd.DataFrame(scaled)
series_origin=series.copy()
series_origin_t=series.copy() 
print(series.values)


(3916, 1)
[[0.34015287]
 [0.32037297]
 [0.30074929]
 ...
 [0.32893241]
 [0.32180583]
 [0.31409749]]


In [105]:
# forecast of N days at a time
for i in range(window_size):
#     print(i)
    series_origin = pd.concat([series_origin, series_origin_t.shift(-(i+1))], axis = 1)
series_origin.dropna(axis=0,inplace=True)  # axis=0 or axis='index’
print(series_origin.shape)   



(3856, 61)


In [106]:
nrow = round(train_sample_fraction*series.shape[0])
print(nrow)
train_days = series.iloc[:nrow, :]
print(train_days.shape)




3133
(3133, 1)


In [107]:
# training data
series_Nday = train_days.copy()
series_Nday_t = train_days.copy() 
for i in range(window_size):
#     print(i)
    series_Nday = pd.concat([series_Nday, series_Nday_t.shift(-(i+1))], axis = 1)
series_Nday.dropna(axis=0,inplace=True)

train_X_Ndays = series_Nday.iloc[:,:-ndays_predict_window_size]
train_y_Ndays = series_Nday.iloc[:,-ndays_predict_window_size:]


train_X_Ndays = train_X_Ndays.values
train_y_Ndays = train_y_Ndays.values

train_X_Ndays = train_X_Ndays.reshape(train_X_Ndays.shape[0],train_X_Ndays.shape[1],1)




(3073, 61)
(3073, 31)
(3073, 30)
(3073, 31, 1)
(3073, 30)


In [108]:
# forcast 1 day at a time
series_1day = train_days.copy()
series_1day_t = train_days.copy() 
for i in range(window_size-ndays_predict_window_size+1):
#     print(i)
    series_1day = pd.concat([series_1day, series_1day_t.shift(-(i+1))], axis = 1)
series_1day.dropna(axis=0,inplace=True)
print(series_1day.shape)  

train_X_1day = series_1day.iloc[:,:-1]
train_y_1day  = series_1day.iloc[:,-1:]

print(train_X_1day.shape)
print(train_y_1day.shape)
train_X_1day  = train_X_1day.values
train_y_1day  = train_y_1day.values

train_X_1day  = train_X_1day.reshape(train_X_1day.shape[0],train_X_1day.shape[1],1)

print(train_X_1day.shape)
print(train_y_1day.shape)




(3102, 32)
(3102, 31)
(3102, 1)
(3102, 31, 1)
(3102, 1)


In [109]:
#Testing data,  Form (None, window, N) for  N days prediction

nrow = round(train_sample_fraction*series.shape[0])
print(nrow)
test = series_origin.iloc[nrow:,:]

print(test.shape)

test_X_Ndays = test.iloc[:,:-ndays_predict_window_size]
test_y_Ndays = test.iloc[:,-ndays_predict_window_size:]

test_X_Ndays = test_X_Ndays.values
test_y_Ndays = test_y_Ndays.values

test_X_Ndays = test_X_Ndays.reshape(test_X_Ndays.shape[0],test_X_Ndays.shape[1],1)

print(test_X_Ndays.shape)
print(test_y_Ndays.shape)

3133
(723, 61)
(723, 31, 1)
(723, 30)


In [110]:
# Testing data,  Form (None, window, 1) for 1 day prediction
nrow = round(train_sample_fraction*series.shape[0])
print(nrow)

test = series_origin.iloc[nrow:,:]


test_X_1day  = test.iloc[:,:-ndays_predict_window_size]
test_y_1day  = test.iloc[:,-ndays_predict_window_size:-ndays_predict_window_size+1]


test_X_1day  = test_X_1day.values
test_y_1day  = test_y_1day.values

test_X_1day = test_X_1day.reshape(test_X_1day.shape[0],test_X_1day.shape[1],1)

print(test_X_1day.shape)
print(test_y_1day.shape)

3133
(723, 31, 1)
(723, 1)


In [ ]:
# TCN
length_input = train_X_Ndays.shape[1]
import tensorflow.keras.layers
from tensorflow.keras import optimizers
# from tensorflow.keras.engine.topology import Layer
from tensorflow.keras.layers import Activation, Lambda
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Activation, multiply

In [ ]:
from tcn import TCN, tcn_full_summary

def prediction_model():
    model = Sequential([

        GRU(16, return_sequences=True, input_shape=(31, 1)),
        Conv1D(filters=16, kernel_size=25, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=16, kernel_size=19, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),

        # TCN层
        TCN(
            nb_filters=16,              # Number of filters
            kernel_size=2,              # Kernel size
            nb_stacks=2,                # Number of stacked TCN layers
            dilations=[1, 2, 4, 8, 16],     # Factor of expansion
            padding='causal',           # Padding
            use_skip_connections=True,  # Skip connections
            dropout_rate=0.2,          # Dropout
            return_sequences=False,     # The output is returned at the last layer or not
            activation='relu',          # Activation function
            kernel_initializer='he_normal'), # kernel initialization
        Dense(30)                      
    ])

    return model

model_TCN = prediction_model()

tcn_full_summary(model_TCN, expand_residual_blocks=False)  # 展开残差块以查看更详细的层信息

In [115]:
# boolean_train_TCN_model=False
if boolean_train_TCN_model:

    model_TCN.compile(loss="mse", optimizer='adam')  # adam
    tensorboard = tensorflow.keras.callbacks.TensorBoard(log_dir="logs/{}".format(time.time()))
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min')
#     model_TCN.summary()


    ########################################### Train LSTM Model ##########################################
    batch_size_lstm = 64
    epoch_lstm = 100
    validation_split_lstm = 0.2
    n_total_samples_lstm = train_X_Ndays.shape[0]
    n_validate_samples_lstm = round(train_X_Ndays.shape[0] * validation_split_lstm)
    n_train_samples_lstm = n_total_samples_lstm - n_validate_samples_lstm
    # save model with best performance during training
    checkpoint = tensorflow.keras.callbacks.ModelCheckpoint(
        resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir +'V_new2new'+str(window_size) +'.h5',
        monitor='val_loss',
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode='auto',
        period=1)
    print('checkpoint = {}'.format(resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir +'V'+str(window_size) +
        '_{epoch:02d}_{val_loss:.6f}' +'.hdf5'))
    start_lstm_training = time.time()
    history = model_TCN.fit(
        train_X_Ndays,
        train_y_Ndays,
        batch_size=batch_size_lstm,
        epochs=epoch_lstm,
        validation_split=validation_split_lstm,
        callbacks=[checkpoint])
    traing_time_lstm = time.time() - start_lstm_training
    print("> LSTM Model Compilation Time : ", traing_time_lstm)
    print('history is {}'.format(history))

    # save training configuration to csv file
    df_lstm_train_conf = pd.DataFrame.from_dict(
        {
            'batch_size': batch_size_lstm,
            'epochs_set': epoch_lstm,
            'epoch_actual': len(history.history['loss']),
            'validation_split': validation_split_lstm,
            'total_samples': n_total_samples_lstm,
            'train_samples': n_train_samples_lstm,
            'validate_samples': n_validate_samples_lstm,
            'training_time': traing_time_lstm
        },
        orient='index')
    df_lstm_train_conf.to_csv(resultsDir_en + TCN_rm_seasonality_dir +
                              each_training_records_dir +
                              'training configurations for LSTM model(V' +
                              str(window_size) + ').csv')
    #df_lstm_train_conf.to_csv(resultsDir_cn+lstm_rm_seasonality_dir+'training configurations for LSTM model.csv')

    # save the model after training specified epochs
    model_TCN.save(resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir +
               'model_V35.h5')
    print('model name = {}'.format(resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir +
               'model_V35.h5'))
    #model.save(resultsDir_cn+lstm_rm_seasonality_dir+'model.h5')

    ########################################## visualize training history #####################################
    ##########  plot LSTM training history (English Version)  #########
    fig, ax = pyplot.subplots(figsize=(6, 6))
    #fig.suptitle('LSTM model loss\n' + location, fontsize=18, fontweight='bold')

    # plot two series
    ax.plot(history.history['loss'], label='train', linewidth=2)
    ax.plot(history.history['val_loss'], label='validate', linewidth=2)

    # set x,y label
    ax.set_xlabel(xlabel='epoch', fontsize=18, fontweight='bold')
    ax.set_ylabel(ylabel='loss', fontsize=18, fontweight='bold')
    # set tick labels
    pyplot.setp(ax.get_xticklabels(), fontsize=15)
    pyplot.setp(ax.get_yticklabels(), fontsize=15)
    # add grid
    #ax.grid()
    # add legend
    legend = ax.legend(loc='upper right', fontsize=15, ncol=1)
    #legend=ax.legend(loc='upper left',fontsize=15, bbox_to_anchor=(0.75, 1.18))

    # save fig
    fig.savefig(
        resultsDir_en + TCN_rm_seasonality_dir + each_training_records_dir + 
        'TCN model training and validation loss(V' +
        str(window_size) + ').eps',
        dpi=300,
        orientation='portrait',
        format='eps',
        bbox_inches='tight')
    pyplot.show()
    
else:  
    model_TCN = load_model('/TCN_rm_seasonality_dir/model_V35.h5',custom_objects={'TCN': TCN})


In [152]:
# prediction for n days

# TCN
preds_Ndays_TCN = model_TCN.predict(test_X_Ndays.reshape(test_X_Ndays.shape[0],test_X_Ndays.shape[1],1))
# inverse transform
preds_Ndays_TCN=scaler.inverse_transform(preds_Ndays_TCN.reshape(preds_Ndays_TCN.shape[0],preds_Ndays_TCN.shape[1]))

print('preds_1day_TCN shape',preds_Ndays_TCN.shape)


preds_1day_TCN shape (723, 30)


In [154]:
def create_colnames(ndays_predict_window_size):
    return ['T+'+str(i+1) for i in range(ndays_predict_window_size)]
print(create_colnames(7))

['T+1', 'T+2', 'T+3', 'T+4', 'T+5', 'T+6', 'T+7']


In [160]:
# R MSE of n days ahead predictions

In [161]:
actuals = test_y_Ndays
actuals = scaler.inverse_transform(actuals)


# Calculate statistics
n_test=actuals.shape[0]
preds_Ndays_TCN=pd.DataFrame(preds_Ndays_TCN)
print('n_test=', n_test)
# TCN
error_TCN_all=np.array([])
rmse_TCN_all=[]
mae_TCN_all=[] # mean absolute error
are_TCN_all=[] # average relative error
r_TCN_all=[]
p_value_TCN_all=[]
NDE_TCN_all=[]

for iDay in range(ndays_predict_window_size):
    iDay_actuals=actuals[:,iDay]   
    # TCN
    iDay_preds_moving_TCN=preds_Ndays_TCN.iloc[:,iDay]

    error_TCN_all=np.append(error_TCN_all,np.subtract(iDay_actuals,iDay_preds_moving_TCN))
    
    rmse_TCN_moving_iDay=math.sqrt(mean_squared_error(iDay_actuals,iDay_preds_moving_TCN))
    rmse_TCN_all.append(rmse_TCN_moving_iDay)
    
    mae_TCN_moving_iDay=sum(abs(np.subtract(iDay_actuals, iDay_preds_moving_TCN)))/iDay_actuals.shape[0]
    mae_TCN_all.append(mae_TCN_moving_iDay)
  
    bbbb_are_TCN_all = abs(np.true_divide(np.subtract(iDay_actuals,iDay_preds_moving_TCN), iDay_actuals ))
    are_TCN_moving_iDay=sum(bbbb_are_TCN_all)/iDay_actuals.shape[0]*100   
    are_TCN_all.append(are_TCN_moving_iDay)
    
    r_TCN_moving_iDay,p_value_TCN_moving_iDay=pearsonr(iDay_actuals,iDay_preds_moving_TCN)
    r_TCN_all.append(r_TCN_moving_iDay)
    p_value_TCN_all.append(p_value_TCN_moving_iDay)
    
    NRE_TCN_moving_iDay=1-sum((iDay_actuals-iDay_preds_moving_TCN)**2)/sum((iDay_actuals-np.mean(iDay_actuals))**2)
    NDE_TCN_all.append(NRE_TCN_moving_iDay)
    

    
    

n_test= 723


In [162]:

# save stastics to csv file
column_names=create_colnames(ndays_predict_window_size)
index_names=['RMSE','MAE','ARE','R','p-value']

# TCN
stat_preds_TCN=[]
stat_preds_TCN.append(rmse_TCN_all)
stat_preds_TCN.append(mae_TCN_all)
stat_preds_TCN.append(are_TCN_all)
stat_preds_TCN.append(r_TCN_all)
stat_preds_TCN.append(p_value_TCN_all)
df_stat_preds_moving_TCN=pd.DataFrame(stat_preds_TCN, columns=column_names,index=index_names)
df_stat_preds_moving_TCN.to_csv(resultsDir_en+TCN_rm_seasonality_dir+'statistics of predictions on different horizons using moving window based on TCN.csv')
df_stat_preds_moving_TCN.to_csv(resultsDir_cn+TCN_rm_seasonality_dir+'statistics of predictions on different horizons using moving window based on TCN.csv')



In [ ]:
# RMSE、MAE、ARE、R和p for 1-14 days predictions
actuals_reshape = actuals.flatten()

preds_Ndays_TCN_reshape = preds_Ndays_TCN.values.flatten()


# RMSE
rmse_TCN_Days=math.sqrt(mean_squared_error(actuals_reshape, preds_Ndays_TCN_reshape))


print("actuals：", type(actuals[1][0]))
# MAE   sum(abs(np.subtract(iDay_actuals,iDay_preds_moving_gb)))/iDay_actuals.shape[0]
mae_TCN_Days=sum(abs(np.subtract(actuals_reshape, preds_Ndays_TCN_reshape)))/actuals_reshape.shape[0]

# ARE   sum(abs(np.subtract(iDay_actuals,iDay_preds_moving_gb)))/iDay_actuals.shape[0]
are_TCN_Days=sum(abs(np.true_divide(np.subtract(actuals_reshape, preds_Ndays_TCN_reshape), actuals_reshape)))/actuals_reshape.shape[0]*100

# R, P
r_TCN_Days,p_TCN_Days=pearsonr(actuals_reshape,preds_Ndays_TCN_reshape)

# NSE

NRE_Proposed_Days=1-sum((actuals_reshape-preds_Ndays_TCN_reshape)**2)/sum((actuals_reshape-np.mean(actuals_reshape))**2)


df_stat_preds_alldays=pd.DataFrame({'RMSE(psu)':{                                           
                                         'TCN':rmse_TCN_Days

                                       },
                               'MAE(psu)':{                                           
                                         'TCN':mae_TCN_Days
                                       },
                               'ARE(%)':{                                           
                                         'TCN':are_TCN_Days
                                       },
                                'R':{                                           
                                         'TCN':r_TCN_Days
                                       },
                                 'P':{                                           
                                         'TCN':p_TCN_Days
                                       },
                                    'NSE':{
                                        'TCN':NRE_Proposed_Days
                                    }
                               })
print('statistics of 14 day ahead prediction using different methods:\n')
print(df_stat_preds_alldays)


# BoxPlot_dir_all = '//boxPlot/5/'
if not os.path.exists(BoxPlot_dir_all):
    os.makedirs(BoxPlot_dir_all)
df_stat_preds_alldays.to_csv(BoxPlot_dir_all+'overall values for 6-30 days.csv')


In [167]:

BoxPlot_dir = BoxPlot_dir_all+'RMSEPoint6/'
if not os.path.exists(BoxPlot_dir):
    os.makedirs(BoxPlot_dir)

df_stat_preds_moving_TCN.to_csv(BoxPlot_dir+'_RMSE_TCN.csv')


In [168]:

RMSE_all_methods = []
MAE_all_methods = []
ARE_all_methods = []
R_all_methods = []
P_all_methods = []
NRE_all_methods = []

# RMSE
RMSE_all_methods.append(rmse_TCN_all)

# MAE
MAE_all_methods.append(mae_TCN_all)

# ARE
ARE_all_methods.append(are_TCN_all)

# R value
R_all_methods.append(r_TCN_all)

# NRE value
NRE_all_methods.append(NDE_TCN_all)

# P value
P_all_methods.append(p_value_TCN_all)

# all metrics
NP_RMSE_all_methods = np.array(RMSE_all_methods)



# if ndays_predict_window_size == 21:
#     days = ['T+1', 'T+2', 'T+3', 'T+4', 'T+5', 'T+6', 'T+7', 'T+8', 'T+9', 'T+10', 'T+11', 'T+12', 'T+13', 'T+14','T+15', 'T+16', 'T+17', 'T+18', 'T+19', 'T+20', 'T+21']
# elif ndays_predict_window_size == 30:
#     days = ['T+1', 'T+2', 'T+3', 'T+4', 'T+5', 'T+6', 'T+7', 'T+8', 'T+9', 'T+10', 'T+11', 'T+12', 'T+13', 'T+14','T+15', 'T+16', 'T+17', 'T+18', 'T+19', 'T+20', 'T+21', 'T+22', 'T+23', 'T+24', 'T+25', 'T+26', 'T+27', 'T+28','T+29','T+30']

days = [n for n in range(ndays_predict_window_size)]

PD_RMSE_all_methods = pd.DataFrame(NP_RMSE_all_methods,index=['TCN'] , columns=days)

PD_MAE_all_methods = pd.DataFrame(NP_MAE_all_methods,index=['TCN'] , columns=days)

PD_ARE_all_methods = pd.DataFrame(NP_ARE_all_methods,index=['TCN'] , columns=days)

PD_R_all_methods = pd.DataFrame(NP_R_all_methods,index=['TCN'] , columns=days)

PD_P_all_methods = pd.DataFrame(NP_P_all_methods,index=['TCN'] , columns=days)


PD_NRE_all_methods = pd.DataFrame(NP_NRE_all_methods,index=['TCN'] , columns=days)

BoxPlot_dir_3 = BoxPlot_dir_all+'/6RMSE、MAE、MAPE、R、p/'
if not os.path.exists(BoxPlot_dir_3):
    os.makedirs(BoxPlot_dir_3)
    
PD_RMSE_all_methods=pd.DataFrame(PD_RMSE_all_methods)
PD_RMSE_all_methods.to_csv(BoxPlot_dir_3+'RMSE.csv', header=True)

PD_MAE_all_methods=pd.DataFrame(PD_MAE_all_methods)
PD_MAE_all_methods.to_csv(BoxPlot_dir_3+'MAE.csv', header=True)

PD_ARE_all_methods=pd.DataFrame(PD_ARE_all_methods)
PD_ARE_all_methods.to_csv(BoxPlot_dir_3+'MAPE.csv', header=True)

PD_R_all_methods=pd.DataFrame(PD_R_all_methods)
PD_R_all_methods.to_csv(BoxPlot_dir_3+'R.csv', header=True)

PD_P_all_methods=pd.DataFrame(PD_P_all_methods)
PD_P_all_methods.to_csv(BoxPlot_dir_3+'P.csv', header=True)

PD_NRE_all_methods=pd.DataFrame(PD_NRE_all_methods)
PD_NRE_all_methods.to_csv(BoxPlot_dir_3+'NRE.csv', header=True)



In [169]:
# error box

In [ ]:
import matplotlib.pyplot as plt

fig,axes=pyplot.subplots(figsize=(9,2.0), dpi=400)


ListError = [error_TCN_all   ]

PD_Error = pd.DataFrame(ListError,index=['Proposed'])



print('PD_Error shape', PD_Error.shape)
PD_T_PD_Error = pd.DataFrame(PD_Error.values.T, index=PD_Error.columns, columns=PD_Error.index)

print('PD_T_PD_Error shape', PD_T_PD_Error.shape)


plt.boxplot(PD_T_PD_Error, showfliers=False, showmeans = True, boxprops=None, labels =['RF', 'SVR', 'BiLSTM', 'Stacked GRU-RNN', 'Proposed'])
axes.set_xlabel(xlabel='',
              fontsize=12,
              fontweight='bold')
axes.set_ylabel(ylabel='Error (psu)',
              fontsize=12,
              fontweight='bold')
# set tick labels
pyplot.setp(axes.get_xticklabels(),fontsize=10,fontweight='bold')
pyplot.setp(axes.get_yticklabels(),fontsize=10,fontweight='bold')

axes.grid()

# legend=axes.legend(loc='best',fontsize=16,ncol=1)



# save fig
fig.savefig(BoxPlot_dir_all+'errorbox.eps',
            dpi=300,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')
pyplot.show()




In [ ]:
# plot and compare statistics of N days ahead prediction using different methods (RMSE)(English Version)
preds_horizons=[(i+1) for i in range(len(rmse_GRU_RNN_all))]

fig,ax=pyplot.subplots(figsize=(6,6))

myInt=1


final_numpy_rmse_TCN_all= np.array(rmse_TCN_all)/myInt


ax.plot(preds_horizons,final_numpy_rmse_TCN_all,color='#000000', marker='d',linestyle='-', label='Proposed')

# set x,y label
ax.set_xlabel(xlabel='prediction horizons (day)',
              fontsize=18,
              fontweight='bold')
ax.set_ylabel(ylabel='RMSE (psu)',
              fontsize=18,
              fontweight='bold')
# set tick labels
pyplot.setp(ax.get_xticklabels(),fontsize=18)
pyplot.setp(ax.get_yticklabels(),fontsize=18)
# add grid
ax.grid()
# add legend
legend=ax.legend(loc='best',fontsize=13,ncol=1)
#legend=ax.legend(loc='upper left',fontsize=15, bbox_to_anchor=(0.825, 1.18))



if not os.path.exists(BoxPlot_dir):
    os.makedirs(BoxPlot_dir)
# save fig
fig.savefig(BoxPlot_dir_all+'RMSE6'+'.eps',
            dpi=300,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')
pyplot.show()




In [172]:
# MAE

In [ ]:
# plot and compare statistics of N days ahead prediction using LSTM and SVR(MAE)(English version)
preds_horizons=[(i+1) for i in range(len(mae_GRU_RNN_all))]

fig,ax=pyplot.subplots(figsize=(6,6))

final_numpy_mae_TCN_all  = np.array(mae_TCN_all)/myInt



ax.plot(preds_horizons,final_numpy_mae_TCN_all,color='#000000', marker='d',linestyle='-', label='Proposed')


# set x,y label
ax.set_xlabel(xlabel='prediction horizons (day)',
              fontsize=18,
              fontweight='bold')
ax.set_ylabel(ylabel='MAE(psu)',
              fontsize=18,
              fontweight='bold')
# set tick labels
pyplot.setp(ax.get_xticklabels(),fontsize=18)
pyplot.setp(ax.get_yticklabels(),fontsize=18)
# add grid
ax.grid()
# add legend
legend=ax.legend(loc='best',fontsize=13,ncol=1)
#legend=ax.legend(loc='upper left',fontsize=15, bbox_to_anchor=(0.825, 1.18))

# save fig
fig.savefig(BoxPlot_dir_all+'MAE6'+'.eps',
            dpi=300,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')
pyplot.show()



In [174]:
# Nash-Sutcliffe Efficiency, NSE

In [ ]:


# plot and compare statistics of N days ahead prediction using LSTM and SVR(p-value)(English version)
preds_horizons=[(i+1) for i in range(len(p_value_GRU_RNN_all))]

fig,ax=pyplot.subplots(figsize=(6,6))

ax.plot(preds_horizons,NDE_TCN_all,color='#000000', marker='d',linestyle='-', label='Proposed', linewidth=2)


# set x,y label
ax.set_xlabel(xlabel='prediction horizons (day)',
              fontsize=18,
              fontweight='bold')
ax.set_ylabel(ylabel='NSE',
              fontsize=18,
              fontweight='bold')

# set axis range
ax.set_ylim([-1,1])

# set tick labels
pyplot.setp(ax.get_xticklabels(),fontsize=18)
pyplot.setp(ax.get_yticklabels(),fontsize=18)
# add grid
ax.grid()
# add legend
legend=ax.legend(loc='best',fontsize=13,ncol=1)
#legend=ax.legend(loc='upper left',fontsize=15, bbox_to_anchor=(0.825, 1.18))

# save fig
fig.savefig(BoxPlot_dir_all+ 'NSE-value_value_point肖2丹4'+'.eps',
            dpi=300,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')
pyplot.show()




In [176]:
# plot actual DET and predicted DET for 1 day ahead (English version)

In [177]:
actuals1day = actuals[:,0]

preds_Ndays_TCN_first1day = preds_Ndays_TCN.values[:,0]



In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(10,7), dpi=300)

n_days=[(i+1) for i in range(len(actuals))]




plt.subplot(5,1,5)
plt.plot(n_days,actuals1day,label='observed',color='g')
plt.plot(n_days,preds_Ndays_TCN_first1day, label='Proposed pred',color='#000000')
# plt.xlabel('(e) Proposed',fontweight='bold',fontsize=14)
plt.tick_params(labelsize=14)
ax=plt.gca();
ax.spines['bottom'].set_linewidth(1.5);
ax.spines['left'].set_linewidth(1.5);
ax.spines['right'].set_linewidth(1.5);
# plt.legend(loc='lower left',fontsize=10, ncol=2)
plt.legend(loc='upper right',fontsize=10, ncol=2)
plt.xlabel('number of days' ,fontweight='bold',fontsize=14)
# ax.set_ylim([33.5,34.5])

plt.tight_layout()

plt.savefig(BoxPlot_dir_all+'prediction_1day'+'.eps',
            dpi=300,
            orientation='portrait',
            format='eps',
            bbox_inches = 'tight')



plt.show()

